In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv("../input/urbansound8k/UrbanSound8K.csv")
df

In [ ]:
df = df[(df.end - df.start) > 0.96*2]
df.reset_index(drop=True, inplace=True)
df

In [ ]:
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

In [ ]:
path ='../input/urbansound8k/'

def embeddingsLabels(df):

    y = []
    emb = []

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
      
        ruta = f"{path}fold{row.fold}/{row.slice_file_name}"
            
        try:
            embeddings = model(ruta)
        except:
            print(ruta)
            break
        
        if len(embeddings.shape) == 1:
            embeddings = embeddings.view(1,-1)
        
        embeddings = torch.mean(embeddings, 0).view(1,-1)
        
        #print(embeddings.shape)        
        emb.append(embeddings.detach().to('cpu').tolist())
        y.append(row["classID"])
        del embeddings
        
        #if index == 10:
           # break
        
    #X = torch.cat(emb,0)
    y = np.array(y)
        
    return emb, y

In [ ]:
X, y = embeddingsLabels(df)

In [ ]:
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)